In [ ]:
import numpy as np
from PIL import Image

In [ ]:
img = Image.open('image.jpg') 
img_array = np.array(img)

In [ ]:
np.save('image.npy', img_array)

In [13]:
import os
import numpy as np
import cv2

base_dir = "CUB_200_2011/images/001.Black_footed_Albatross"
data = []
labels = []


In [12]:

# loop through each folder in the base directory
folder_path = os.path.join(base_dir)

if os.path.isdir(folder_path):
    # loop through each image in the folder
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)

            # read the image using OpenCV
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # assuming grayscale, change if you want color
        if img is not None:
                # resize to a standard size if needed (optional)
            img = cv2.resize(img, (64, 64))  # replace with desired size

                # append the image data and label to our lists
            data.append(img)
            labels.append(folder_name)



NameError: name 'folder_name' is not defined

In [14]:
folder_path

'CUB_200_2011/images/001.Black_footed_Albatross'

In [16]:
for img_name in os.listdir(folder_path):
    img_path = os.path.join(folder_path, img_name)

        # read the image using OpenCV
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # assuming grayscale, change if you want color
    if img is not None:
            # resize to a standard size if needed (optional)
        img = cv2.resize(img, (64, 64))  # replace with desired size

            # append the image data and label to our lists
        data.append(img)


In [ ]:
# convert lists to numpy arrays
data_np = np.array(data, dtype=np.float32) / 255.0  # normalize to [0, 1]
labels_np = np.array(labels)

# save to npy files
np.save("data.npy", data_np)
np.save("labels.npy", labels_np)
